**Purpose**

Carry out additional processing and analyses related to:
* Describing contributions to $\Delta$ Context scores by position
* Determining mutational signatures where constraint correlates with $\Delta$ Context score
* Determining thresholds for $\Delta$ Context scores that select for variants unobserved in gnomAD

# Setup

In [1]:
import pandas as pd
import numpy as np
import glob

In [2]:
from statsmodels.stats.multitest import fdrcorrection

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
import importlib.util

In [5]:
ccv_spec = importlib.util.spec_from_file_location("codon_context_variables", "../codon_context_variables.py")
ccv = importlib.util.module_from_spec(ccv_spec)
ccv_spec.loader.exec_module(ccv)


## Files - read

Variants annotated with context scores

In [6]:
#Per AA files with per-position context score values
variant_scored_ann_files = glob.glob("../../data/3_codon_context_score/syn_variant_codon_context_score_byAminoAcid_sub/ssnv_codon_context_score_mi_annotated_12nt_AminoAcid*_seqCol.tsv")
#Combined table with gnomAD and structural annotations
variant_scored_table_file = "../../data/3_codon_context_score/syn_variant_12nt_codon_context_score_CP3_zCodon.tsv"

Constraint curve line fits data

In [7]:
constraint_curve_filename = "../../data/3_codon_context_score/constraint_curve_fits_1000min_dsCS_RCodonZ_100b_12nt_CP3_xSNVContext_y.tsv"
constraint_curve_rand_filename = "../../data/3_codon_context_score/constraint_curve_fits_1000min_dsCS_RCodonZ_100b_12nt_CP3_xSNVContext_yrand.tsv"

Score thresholding

In [8]:
constraint_thresholds_posScaled_filename = "../../data/3_codon_context_score/context_score_threshold_quantile_xSNVContext_diff_sum_context_score_posScaled.tsv"

## Files - write

Context score difference by position group

In [9]:
variant_weight_xPositionGroup_xSNVContext_file = "../../data/3_codon_context_score/syn_variant_12nt_codon_context_scoreAvg_xPosition_xSNVContext.tsv"

Summary of curve fits to $\Delta$ Context score vs. probability of being observed in gnomAD, by SNVContext

In [10]:
constraint_curve_summary_file = "../../data/3_codon_context_score/constraint_curve_fits_1000min_dsCS_RCodonZ_100b_12nt_CP3_xSNVContext_y_yrand_summary.tsv"

## Functions

In [11]:
def get_position_label (x) :
    
    y = int(x.split("_")[0][1:])
    if y < 0 :
        z = abs(y)
        
        if z <= 3 :
            return "N-"+str(z)
        elif z%3 == 0:
            return "CP1"
        elif z%3 == 2:
            return "CP2"
        elif z%3 == 1 :
            return "CP3"
        else :
            print("What position?", y)
    else :
        z = y
        if z <= 3 :
            return "N+"+str(z)
        elif z%3 == 0 :
            return "CP3"
        elif z%3 == 2:
            return "CP2"
        elif z%3 == 1:
            return "CP1"
        else :
            print("What position?", y)
            
    return 0

In [12]:
def abs_mean(x) :
    result = np.mean([abs(y) for y in x])
    
    return result

# Combining annotated context score tables

* Read in select columns from combined table, filter for gnomAD status
* Read in annotated, per-AA files, and concatenate
* Concatenate

In [13]:
#Read in combined table
variant_scored_df = pd.read_csv(variant_scored_table_file,
                                sep="\t",
                                usecols=["CHR", "POS", "REF", "ALT",
                                         "REF_AminoAcid", "REF_AminoAcid_sub",
                                         "REF_Codon", "ALT_Codon", "CodonChangeSig",
                                         "NM_ID", "entrezgene", "name",
                                         "y",
                                         "REF_efeValue", "ALT_efeValue", "deltaEFE",
                                         "diff_sum_context_score_REF_Codon_zscore"],
                                dtype={"CHR":str})
variant_scored_df.head()

,CHR,POS,REF,ALT,NM_ID,deltaEFE,REF_efeValue,ALT_efeValue,entrezgene,name,REF_AminoAcid,REF_AminoAcid_sub,REF_Codon,ALT_Codon,diff_sum_context_score_REF_Codon_zscore,y,CodonChangeSig
0,5,149981141,T,C,NM_000112.3,-1.28,-20.70,-21.98,1836,solute carrier family 26 member 2,F,F,TTT,TTC,-0.808288,0,TTT>TTC
1,5,149977859,T,C,NM_000112.3,3.58,-18.91,-15.33,1836,solute carrier family 26 member 2,F,F,TTT,TTC,-1.007113,0,TTT>TTC
2,5,149977928,C,T,NM_000112.3,-1.12,-21.72,-22.84,1836,solute carrier family 26 member 2,F,F,TTC,TTT,1.307013,0,TTC>TTT
3,5,149981087,T,C,NM_000112.3,0.56,-22.99,-22.43,1836,solute carrier family 26 member 2,F,F,TTT,TTC,0.659425,1,TTT>TTC
4,5,149980883,C,T,NM_000112.3,-1.64,-16.61,-18.25,1836,solute carrier family 26 member 2,F,F,TTC,TTT,-0.532575,0,TTC>TTT


In [14]:
variant_scored_df["y"].value_counts()

 0    12916367
-1     4511803
 1     2267089
Name: y, dtype: int64

In [15]:
#Keep variants with gnomAD coverage
variant_scored_gnomad_df = variant_scored_df.query("y != -1").copy()
del variant_scored_df

In [16]:
variant_scored_gnomad_df.shape

(15183456, 17)

In [17]:
#Read in per-AA files with extra annotations 
# and merge with table of gnomAD variants
variant_scored_gnomad_ann_dfs = []
variant_scored_gnomad_xAA = variant_scored_gnomad_df.groupby("REF_AminoAcid_sub")

for amin, vsg_amin_df in variant_scored_gnomad_xAA :
    variant_scored_ann_amin = ("../../data/3_codon_context_score/syn_variant_codon_context_score_byAminoAcid_sub/ssnv_codon_context_score_mi_annotated_12nt_AminoAcid"+ 
                               amin+"_seqCol.tsv")
    print("Reading", variant_scored_ann_amin)
    
    variant_scored_ann_amin_df = pd.read_csv(variant_scored_ann_amin,
                                             sep="\t",
                                             dtype={"CHR":str})
    vsg_amin_concat_df = vsg_amin_df.merge(variant_scored_ann_amin_df,
                                           on=["CHR", "POS", "REF", "ALT",
                                               "REF_AminoAcid", "REF_AminoAcid_sub",
                                               "REF_Codon", "ALT_Codon"],
                                           how="left")
    variant_scored_gnomad_ann_dfs.append(vsg_amin_concat_df)
    
    print(vsg_amin_concat_df.shape)
    

Reading ../../data/3_codon_context_score/syn_variant_codon_context_score_byAminoAcid_sub/ssnv_codon_context_score_mi_annotated_12nt_AminoAcidA_seqCol.tsv
(1552642, 133)
Reading ../../data/3_codon_context_score/syn_variant_codon_context_score_byAminoAcid_sub/ssnv_codon_context_score_mi_annotated_12nt_AminoAcidC_seqCol.tsv
(181087, 133)
Reading ../../data/3_codon_context_score/syn_variant_codon_context_score_byAminoAcid_sub/ssnv_codon_context_score_mi_annotated_12nt_AminoAcidD_seqCol.tsv
(383662, 133)
Reading ../../data/3_codon_context_score/syn_variant_codon_context_score_byAminoAcid_sub/ssnv_codon_context_score_mi_annotated_12nt_AminoAcidE_seqCol.tsv
(559377, 133)
Reading ../../data/3_codon_context_score/syn_variant_codon_context_score_byAminoAcid_sub/ssnv_codon_context_score_mi_annotated_12nt_AminoAcidF_seqCol.tsv
(297508, 133)
Reading ../../data/3_codon_context_score/syn_variant_codon_context_score_byAminoAcid_sub/ssnv_codon_context_score_mi_annotated_12nt_AminoAcidG_seqCol.tsv
(1480

In [19]:
variant_scored_gnomad_ann_df = pd.concat(variant_scored_gnomad_ann_dfs,
                                         axis=0)
print(variant_scored_gnomad_ann_df.shape)

(15183456, 133)


# Position representation in scores

How much do bias factors originating from each main region of the sequence context contribute to the overall sequence context score?

* Separate out the nearest 3 upstream and 3 downstream positions $(-3, -2, -1, +1, +2, +3)$
* Outside that range, group the positions at CP1 sites, CP2 sites, CP3 sites to sum along
* For each of these categories, add up the absolute value of their total contribution: $|s(c_{ALT},.)-s(c_{REF},.)|$

In [22]:
variant_scored_gnomad_pos_fraction = variant_scored_gnomad_ann_df.copy()
variant_scored_gnomad_pos_fraction["CP1_sum"] = (variant_scored_gnomad_pos_fraction["N-12_diff_score"]+
                                                 variant_scored_gnomad_pos_fraction["N-9_diff_score"]+
                                                 variant_scored_gnomad_pos_fraction["N-6_diff_score"]+
                                                 variant_scored_gnomad_pos_fraction["N+4_diff_score"]+
                                                 variant_scored_gnomad_pos_fraction["N+7_diff_score"]+
                                                 variant_scored_gnomad_pos_fraction["N+10_diff_score"])
variant_scored_gnomad_pos_fraction["CP2_sum"] = (variant_scored_gnomad_pos_fraction["N-11_diff_score"]+
                                                 variant_scored_gnomad_pos_fraction["N-8_diff_score"]+
                                                 variant_scored_gnomad_pos_fraction["N-5_diff_score"]+
                                                 variant_scored_gnomad_pos_fraction["N+5_diff_score"]+
                                                 variant_scored_gnomad_pos_fraction["N+8_diff_score"]+
                                                 variant_scored_gnomad_pos_fraction["N+11_diff_score"])
variant_scored_gnomad_pos_fraction["CP3_sum"] = (variant_scored_gnomad_pos_fraction["N-10_diff_score"]+
                                                 variant_scored_gnomad_pos_fraction["N-7_diff_score"]+
                                                 variant_scored_gnomad_pos_fraction["N-4_diff_score"]+
                                                 variant_scored_gnomad_pos_fraction["N+6_diff_score"]+
                                                 variant_scored_gnomad_pos_fraction["N+9_diff_score"]+
                                                 variant_scored_gnomad_pos_fraction["N+12_diff_score"])

variant_scored_gnomad_pos_fraction["sum_abs_total"] = (variant_scored_gnomad_pos_fraction["N-3_diff_score"].apply(abs)+
                                                 variant_scored_gnomad_pos_fraction["N-2_diff_score"].apply(abs)+
                                                 variant_scored_gnomad_pos_fraction["N-1_diff_score"].apply(abs)+
                                                 variant_scored_gnomad_pos_fraction["N+1_diff_score"].apply(abs)+
                                                 variant_scored_gnomad_pos_fraction["N+2_diff_score"].apply(abs)+
                                                 variant_scored_gnomad_pos_fraction["N+3_diff_score"].apply(abs)+
                                                 variant_scored_gnomad_pos_fraction["CP1_sum"].apply(abs)+
                                                 variant_scored_gnomad_pos_fraction["CP2_sum"].apply(abs)+
                                                 variant_scored_gnomad_pos_fraction["CP3_sum"].apply(abs))

Take the absolute value of each category and compare to the total

In [23]:
mod_columns = ["N-3_diff_score","N-2_diff_score","N-1_diff_score",
               "N+1_diff_score","N+2_diff_score","N+3_diff_score",
               "CP1_sum","CP2_sum","CP3_sum"]
new_columns = []
for column_ in mod_columns :
     new_column = column_.split("_")[0]+"_absFrac"
     new_columns.append(new_column)
     
     variant_scored_gnomad_pos_fraction[new_column] = (variant_scored_gnomad_pos_fraction[column_].apply(abs)/
                                                 variant_scored_gnomad_pos_fraction["sum_abs_total"])

variant_scored_gnomad_pos_fraction.head()

,CHR,POS,REF,ALT,NM_ID,deltaEFE,REF_efeValue,ALT_efeValue,entrezgene,name,REF_AminoAcid,REF_AminoAcid_sub,REF_Codon,ALT_Codon,diff_sum_context_score_REF_Codon_zscore,y,CodonChangeSig,Sense,ALT_AminoAcid,ALT_AminoAcid_sub,CodonPosition,WindowPosition,PrecedingBase,TrailingBase,PrecedingBases,TrailingBases,SNVContext,TribaseContext,PrecedingCodon,TrailingCodon,PrecedingBicodon,TrailingBicodon,TricodonContext,REF_Sequence,N-12,N-11,N-10,N-9,N-8,N-7,N-6,N-5,N-4,N-3,N-2,N-1,N+1,N+2,N+3,N+4,N+5,N+6,N+7,N+8,N+9,N+10,N+11,N+12,N-12_ref_score,N-12_alt_score,N-11_ref_score,N-11_alt_score,N-10_ref_score,N-10_alt_score,N-9_ref_score,N-9_alt_score,N-8_ref_score,N-8_alt_score,N-7_ref_score,N-7_alt_score,N-6_ref_score,N-6_alt_score,N-5_ref_score,N-5_alt_score,N-4_ref_score,N-4_alt_score,N-3_ref_score,N-3_alt_score,N-2_ref_score,N-2_alt_score,N-1_ref_score,N-1_alt_score,N+1_ref_score,N+1_alt_score,N+2_ref_score,N+2_alt_score,N+3_ref_score,N+3_alt_score,N+4_ref_score,N+4_alt_score,N+5_ref_score,N+5_alt_score,N+6_ref_score,N+6_alt_score,N+7_ref_score,N+7_alt_score,N+8_ref_score,N+8_alt_score,N+9_ref_score,N+9_alt_score,N+10_ref_score,N+10_alt_score,N+11_ref_score,N+11_alt_score,N+12_ref_score,N+12_alt_score,N-12_diff_score,N-11_diff_score,N-10_diff_score,N-9_diff_score,N-8_diff_score,N-7_diff_score,N-6_diff_score,N-5_diff_score,N-4_diff_score,N-3_diff_score,N-2_diff_score,N-1_diff_score,N+1_diff_score,N+2_diff_score,N+3_diff_score,N+4_diff_score,N+5_diff_score,N+6_diff_score,N+7_diff_score,N+8_diff_score,N+9_diff_score,N+10_diff_score,N+11_diff_score,N+12_diff_score,sum_ref_context_score,sum_alt_context_score,diff_sum_context_score,CP1_sum,CP2_sum,CP3_sum,sum_abs_total,N-3_absFrac,N-2_absFrac,N-1_absFrac,N+1_absFrac,N+2_absFrac,N+3_absFrac,CP1_absFrac,CP2_absFrac,CP3_absFrac
0,5,149981579,A,C,NM_000112.3,2.01,-16.12,-14.11,1836,solute carrier family 26 member 2,A,A,GCA,GCC,-1.531783,0,GCA>GCC,+,A,A,3,51,C,G,AGC,GGG,A>C,CAG,ACA,GGG,ACAGCA,GCAGGG,ACAGCAGGG,CTGCATACTATAGTGATTGACTGCAGTGCAATTCAATTTTTAGATA...,T,T,T,T,T,A,G,A,T,A,C,A,G,G,G,A,T,C,C,A,C,A,C,A,0.002026,-0.002113,0.001681,-0.001315,0.009777,-0.011262,0.001152,-0.001131,0.000625,-0.000301,0.011662,-0.012458,-0.004174,0.001884,0.006374,-0.005621,0.007637,-0.010203,0.006239,-0.003463,-0.001443,0.000109,0.014856,-0.011619,0.031763,-0.049864,-0.002963,0.004877,-0.009482,0.011324,0.007605,-0.005246,0.000561,-0.001316,-0.013378,0.022121,-0.005884,0.009755,0.003545,-0.003586,-0.012269,0.020416,0.005099,-0.004421,-0.002238,0.002074,0.013537,-0.013044,-0.004140,-0.002996,-0.021039,-0.002283,-0.000926,-0.024120,0.006058,-0.011995,-0.017840,-0.009703,0.001552,-0.026475,-0.081627,0.007840,0.020805,-0.012852,-0.001877,0.035499,0.015640,-0.007131,0.032685,-0.009521,0.004312,-0.026581,0.072309,-0.064404,-0.136713,-0.007097,-0.020612,-0.021396,0.197107,0.049225,0.007873,0.134319,0.414125,0.039774,0.105553,0.036004,0.104575,0.108552
1,5,149980400,C,G,NM_000112.3,1.37,-25.31,-23.94,1836,solute carrier family 26 member 2,A,A,GCC,GCG,0.141391,0,GCC>GCG,+,A,A,3,51,C,A,GGC,AAG,C>G,CCA,CAG,AAG,CAGGCC,GCCAAG,CAGGCCAAG,CTGAGTGGATTTGTCACTGGTGCCTCCTTCACTATTCTTACATCTC...,C,T,T,A,C,A,T,C,T,C,A,G,A,A,G,T,A,T,C,T,T,C,T,T,0.005398,0.003874,-0.001315,-0.001287,-0.011262,-0.006134,-0.004350,-0.004281,0.002786,0.002217,-0.012458,-0.005255,-0.002573,-0.002146,0.004727,0.000729,-0.010203,-0.006589,0.004351,0.003897,-0.002943,-0.002083,0.010983,0.010195,0.055177,-0.008539,-0.005341,-0.004402,0.011324,0.010064,-0.000743,-0.001997,-0.002065,-0.004637,-0.012964,-0.005873,0.009755,0.002033,0.001272,-0.002459,-0.011476,-0.006198,0.006666,0.003156,0.000876,-0.002074,-0.011384,-0.006036,-0.001524,0.000028,0.005127,0.000069,-0.000569,0.007203,0.000427,-0.003998,0.003614,-0.000454,0.000860,-0.000789,-0.063716,0.000939,-0.001259,-0.001254,-0.002572,0.007091,-0.007722,-0.003731,0.005277,-0.003510,-0.002949,0.005348,0.024239,-0.033826,-0.058065,-0.013515,-0.013792,0.033661,0.128985,0.003520,0.006666,0.006115,0.493979,0.007281,0.009764,0.1

In [24]:
#Group by REF>ALT change and take mean across records
variant_scored_gnomad_pos_fraction_mean = variant_scored_gnomad_pos_fraction.\
    groupby("SNVContext").\
        agg({x:np.mean for x in new_columns}).\
            rename(columns={x:x.split("_")[0] for x in new_columns})

variant_scored_gnomad_pos_fraction_mean.head()

,N-3,N-2,N-1,N+1,N+2,N+3,CP1,CP2,CP3
SNVContext,,,,,,,,,
A>C,0.034414,0.031215,0.114263,0.219819,0.071526,0.129277,0.074541,0.065845,0.259101
A>G,0.050534,0.031320,0.146780,0.099379,0.041283,0.145961,0.105490,0.082739,0.296515
A>T,0.061794,0.067946,0.100961,0.196720,0.145821,0.068686,0.094084,0.092512,0.171478
C>A,0.031293,0.029810,0.118025,0.180915,0.070928,0.131848,0.072687,0.063796,0.300698
C>G,0.031973,0.052842,0.078497,0.298193,0.090318,0.098623,0.076950,0.053684,0.218920


Save summary table

In [25]:
variant_scored_gnomad_pos_fraction_mean.to_csv(variant_weight_xPositionGroup_xSNVContext_file,
                                               sep="\t",
                                               index=True)

# Determining constrained contexts

See where:
* Linear coefficient is statistically significant
* Linear coefficient measured on shuffled data is not statistically significant or absolute value of coefficient is less than that for observed data

In [26]:
constraint_curve_df = pd.read_csv(constraint_curve_filename,
                                  sep="\t").\
                                      set_index("group_label")
constraint_curve_df.head()

,group_variable,y_variable,r1_x0,r1_x1,r1_x0_pv,r1_x1_pv,r1_rsq_adj
group_label,,,,,,,
A>C,SNVContext,y_mean,0.055544,0.002346,2.683252e-119,1.044242e-17,0.524170
A>G,SNVContext,y_mean,0.179461,-0.001188,3.674384e-143,6.461575e-02,0.024563
A>T,SNVContext,y_mean,0.040679,0.019842,3.394397e-74,1.132109e-24,0.795802
C>A,SNVContext,y_mean,0.071544,0.006183,6.206382e-131,1.645867e-36,0.802516
C>G,SNVContext,y_mean,0.084596,0.035563,2.788518e-83,7.788825e-39,0.838708


In [27]:
constraint_curve_rand_df = pd.read_csv(constraint_curve_rand_filename,
                                       sep="\t").\
                                           set_index("group_label")
constraint_curve_rand_df.head()

,group_variable,y_variable,r1_x0,r1_x1,r1_x0_pv,r1_x1_pv,r1_rsq_adj
group_label,,,,,,,
A>C,SNVContext,y_rand_mean,0.054805,-0.000084,4.952189e-131,0.616270,-0.007605
A>G,SNVContext,y_rand_mean,0.179474,-0.000855,7.513538e-142,0.195030,0.007045
A>T,SNVContext,y_rand_mean,0.042326,0.000267,4.422485e-96,0.653850,-0.012040
C>A,SNVContext,y_rand_mean,0.070809,0.000589,9.988872e-149,0.004109,0.071611
C>G,SNVContext,y_rand_mean,0.095972,0.001122,8.427440e-126,0.078790,0.022450


Set p-value threshold: 14 contexts, 2 data sets, 2 directions

In [28]:
pval_threshold = 0.05/(14*2*2)
pval_threshold

0.0008928571428571429

In [29]:
snvcon_constrained = []

#Loop through SNVContexts
for snvcon in constraint_curve_df.index :
    print("SNVContext:", snvcon)
    
    #Check if p-value of linear term meets threshold
    if (constraint_curve_df.loc[snvcon, "r1_x1_pv"] < pval_threshold) :
        #Check if corresponding term from shuffled data set meets threshold
        if (constraint_curve_rand_df.loc[snvcon, "r1_x1_pv"] < pval_threshold) :
            #Compare magnitude of slopes
            if (abs(constraint_curve_df.loc[snvcon, "r1_x1"]) > 
                abs(constraint_curve_rand_df.loc[snvcon, "r1_x1"])) :
                print("+Stat. sig. linear slope but Observed effect greater than Shuffled effect")
                snvcon_constrained.append(snvcon)
            else :
                print("-Stat. sig. linear slope but Observed effect not greater than Shuffled effect")
        else :
            print("+Stat. sig. linear slope in Observed data only")
            snvcon_constrained.append(snvcon)
    else :
        #Check if p-value of linear term from the shuffled data set instead meets the threshold
        if (constraint_curve_rand_df.loc[snvcon, "r1_x1_pv"] < pval_threshold) :
            print("-Stat. sig. linear slope in Shuffled data only")
        else :
            print("-No stat. sig. linear slope")
                

SNVContext: A>C
+Stat. sig. linear slope in Observed data only
SNVContext: A>G
-No stat. sig. linear slope
SNVContext: A>T
+Stat. sig. linear slope in Observed data only
SNVContext: C>A
+Stat. sig. linear slope in Observed data only
SNVContext: C>G
+Stat. sig. linear slope in Observed data only
SNVContext: C>T
+Stat. sig. linear slope in Observed data only
SNVContext: CpG>CpA
+Stat. sig. linear slope in Observed data only
SNVContext: CpG>TpG
+Stat. sig. linear slope but Observed effect greater than Shuffled effect
SNVContext: G>A
+Stat. sig. linear slope in Observed data only
SNVContext: G>C
+Stat. sig. linear slope but Observed effect greater than Shuffled effect
SNVContext: G>T
-No stat. sig. linear slope
SNVContext: T>A
-No stat. sig. linear slope
SNVContext: T>C
-Stat. sig. linear slope in Shuffled data only
SNVContext: T>G
-No stat. sig. linear slope


In [30]:
#Contexts with significant relationship measured
snvcon_constrained

['A>C', 'A>T', 'C>A', 'C>G', 'C>T', 'CpG>CpA', 'CpG>TpG', 'G>A', 'G>C']

In [31]:
constraint_curve_df.loc[snvcon_constrained]

,group_variable,y_variable,r1_x0,r1_x1,r1_x0_pv,r1_x1_pv,r1_rsq_adj
group_label,,,,,,,
A>C,SNVContext,y_mean,0.055544,0.002346,2.683252e-119,1.044242e-17,0.524170
A>T,SNVContext,y_mean,0.040679,0.019842,3.394397e-74,1.132109e-24,0.795802
C>A,SNVContext,y_mean,0.071544,0.006183,6.206382e-131,1.645867e-36,0.802516
C>G,SNVContext,y_mean,0.084596,0.035563,2.788518e-83,7.788825e-39,0.838708
C>T,SNVContext,y_mean,0.233480,-0.010381,2.710118e-173,2.062009e-46,0.875886
CpG>CpA,SNVContext,y_mean,0.873599,0.058283,4.933596e-158,1.223978e-45,0.873999
CpG>TpG,SNVContext,y_mean,0.848329,0.049285,1.183826e-156,7.751373e-63,0.971198
G>A,SNVContext,y_mean,0.214145,-0.005665,1.831090e-178,1.012139e-29,0.728564
G>C,SNVContext,y_mean,0.075291,0.019929,6.661412e-124,2.177913e-63,0.947335


In [15]:
constraint_curve_rand_df.loc[snvcon_constrained]

,group_variable,y_variable,r1_x0,r1_x1,r1_x0_pv,r1_x1_pv,r1_rsq_adj,r2_x0,r2_x1,r2_x2,r2_x0_pv,r2_x1_pv,r2_x2_pv,r2_rsq_adj
group_label,,,,,,,,,,,,,,
A>C,SNVContext,y_rand_mean,0.054805,-0.000084,4.952189e-131,6.162703e-01,-0.007605,0.054561,0.000204,0.000131,2.109900e-126,3.504721e-01,4.445047e-02,0.023738
A>T,SNVContext,y_rand_mean,0.042326,0.000267,4.422485e-96,6.538498e-01,-0.012040,0.042410,0.000420,-0.000908,3.902411e-90,5.111326e-01,4.933944e-01,-0.020163
C>A,SNVContext,y_rand_mean,0.070809,0.000589,9.988872e-149,4.109103e-03,0.071611,0.070514,0.000798,0.000252,3.453768e-144,2.253132e-04,6.780802e-03,0.130646
C>G,SNVContext,y_rand_mean,0.095972,0.001122,8.427440e-126,7.878996e-02,0.022450,0.094794,-0.002265,0.004288,2.760817e-133,3.242571e-04,8.190619e-14,0.462652
C>T,SNVContext,y_rand_mean,0.235129,-0.000884,1.209512e-187,2.266055e-03,0.081889,0.235359,-0.001066,-0.000229,7.532822e-182,6.516636e-04,1.167780e-01,0.095771
CpG>CpA,SNVContext,y_rand_mean,0.866781,-0.001263,8.541578e-196,1.654813e-01,0.009628,0.865397,-0.001359,0.001448,2.203473e-183,1.316875e-01,6.073931e-02,0.035494
CpG>TpG,SNVContext,y_rand_mean,0.879883,0.003139,1.042027e-162,2.778560e-04,0.144214,0.879907,0.001067,0.001211,4.943302e-161,5.505763e-01,1.933821e-01,0.151956
G>A,SNVContext,y_rand_mean,0.214085,-0.000324,3.200964e-180,3.330369e-01,-0.000542,0.213121,-0.000270,0.000998,1.804919e-175,3.672037e-01,2.530955e-06,0.196386
G>C,SNVContext,y_rand_mean,0.079350,0.006872,1.549756e-136,1.865876e-33,0.779063,0.080214,0.007710,-0.001511,4.914570e-132,1.206337e-35,2.200715e-05,0.815469


Make and save joint table with constrained contexts flagged:

In [32]:
constraint_curve_refseq_shuff_df = pd.merge(constraint_curve_df,
                                            constraint_curve_rand_df,
                                            how="inner",
                                            left_index=True,
                                            right_index=True,
                                            suffixes=("_ref","_shuff"))
constraint_curve_refseq_shuff_df.head()

,group_variable_ref,y_variable_ref,r1_x0_ref,r1_x1_ref,r1_x0_pv_ref,r1_x1_pv_ref,r1_rsq_adj_ref,group_variable_shuff,y_variable_shuff,r1_x0_shuff,r1_x1_shuff,r1_x0_pv_shuff,r1_x1_pv_shuff,r1_rsq_adj_shuff
group_label,,,,,,,,,,,,,,
A>C,SNVContext,y_mean,0.055544,0.002346,2.683252e-119,1.044242e-17,0.524170,SNVContext,y_rand_mean,0.054805,-0.000084,4.952189e-131,0.616270,-0.007605
A>G,SNVContext,y_mean,0.179461,-0.001188,3.674384e-143,6.461575e-02,0.024563,SNVContext,y_rand_mean,0.179474,-0.000855,7.513538e-142,0.195030,0.007045
A>T,SNVContext,y_mean,0.040679,0.019842,3.394397e-74,1.132109e-24,0.795802,SNVContext,y_rand_mean,0.042326,0.000267,4.422485e-96,0.653850,-0.012040
C>A,SNVContext,y_mean,0.071544,0.006183,6.206382e-131,1.645867e-36,0.802516,SNVContext,y_rand_mean,0.070809,0.000589,9.988872e-149,0.004109,0.071611
C>G,SNVContext,y_mean,0.084596,0.035563,2.788518e-83,7.788825e-39,0.838708,SNVContext,y_rand_mean,0.095972,0.001122,8.427440e-126,0.078790,0.022450


In [33]:
#Add indicator column
constraint_curve_refseq_shuff_ni_df = constraint_curve_refseq_shuff_df.reset_index()
constraint_curve_refseq_shuff_ni_df["constrained_sig"] = constraint_curve_refseq_shuff_ni_df["group_label"].\
    apply(lambda x: 1 if x in snvcon_constrained else 0)
constraint_curve_refseq_shuff_ni_df.head()

,group_label,group_variable_ref,y_variable_ref,r1_x0_ref,r1_x1_ref,r1_x0_pv_ref,r1_x1_pv_ref,r1_rsq_adj_ref,group_variable_shuff,y_variable_shuff,r1_x0_shuff,r1_x1_shuff,r1_x0_pv_shuff,r1_x1_pv_shuff,r1_rsq_adj_shuff,constrained_sig
0,A>C,SNVContext,y_mean,0.055544,0.002346,2.683252e-119,1.044242e-17,0.524170,SNVContext,y_rand_mean,0.054805,-0.000084,4.952189e-131,0.616270,-0.007605,1
1,A>G,SNVContext,y_mean,0.179461,-0.001188,3.674384e-143,6.461575e-02,0.024563,SNVContext,y_rand_mean,0.179474,-0.000855,7.513538e-142,0.195030,0.007045,0
2,A>T,SNVContext,y_mean,0.040679,0.019842,3.394397e-74,1.132109e-24,0.795802,SNVContext,y_rand_mean,0.042326,0.000267,4.422485e-96,0.653850,-0.012040,1
3,C>A,SNVContext,y_mean,0.071544,0.006183,6.206382e-131,1.645867e-36,0.802516,SNVContext,y_rand_mean,0.070809,0.000589,9.988872e-149,0.004109,0.071611,1
4,C>G,SNVContext,y_mean,0.084596,0.035563,2.788518e-83,7.788825e-39,0.838708,SNVContext,y_rand_mean,0.095972,0.001122,8.427440e-126,0.078790,0.022450,1


In [34]:
constraint_curve_refseq_shuff_ni_df.to_csv(constraint_curve_summary_file,
                                           sep="\t",
                                           index=True)

# Choosing threshold for context score

Go through assessment metrics for using $\Delta$ context score as a classifier for variants observed vs. unobserved in gnomAD.

In [35]:
threshold_ps_df = pd.read_csv(constraint_thresholds_posScaled_filename,
                           sep="\t")
threshold_ps_df.head()

,group,threshold_index,threshold_quantile,threshold_value,comparison,selected_total,selected_0,selected_1,overall_0,overall_1,frac0_on_selected,frac0_on_unselected,frac0_overall,sensitivity,specificity,test_stat,test_pv
0,A>C,1,0.01,-3.080014,above,847488,800986,46502,809111,46938,0.945130,0.949071,0.945169,0.989958,0.009289,2.541003,0.110924
1,A>C,1,0.01,-3.080014,below,8561,8125,436,809111,46938,0.949071,0.945130,0.945169,0.010042,0.990711,2.541003,0.110924
2,A>C,2,0.02,-2.784060,above,838928,792854,46074,809111,46938,0.945080,0.949536,0.945169,0.979908,0.018407,6.427775,0.011235
3,A>C,2,0.02,-2.784060,below,17121,16257,864,809111,46938,0.949536,0.945080,0.945169,0.020092,0.981593,6.427775,0.011235
4,A>C,3,0.03,-2.584174,above,830367,784704,45663,809111,46938,0.945009,0.950354,0.945169,0.969835,0.027163,13.736337,0.000210


In [36]:
#Remove 0 values
threshold_ps_nz_df = threshold_ps_df[threshold_ps_df["threshold_index"]<100].copy()

In [37]:
threshold_ps_nz_df.shape

(2772, 17)

In [38]:
#Consider SNVContexts where a significant relationship was already observed
snvcon_constrained

['A>C', 'A>T', 'C>A', 'C>G', 'C>T', 'CpG>CpA', 'CpG>TpG', 'G>A', 'G>C']

Apply FDR correction to p-value of test statistic measuring enrichment of variants unobserved in gnomAD

In [39]:
rej_array, threshold_ps_nz_df["test_pv_adj"] = \
    fdrcorrection(threshold_ps_nz_df["test_pv"],
                  alpha=0.05,
                  method='indep',
                  is_sorted=False)

In [40]:
#Specify minimum specificity of classification and p-value for enrichment test
spec_min = 0.95
adj_pv_max = 0.05

In [42]:
#Apply thresholds
spec_filter = "specificity > "+str(spec_min)
pv_filter = "test_pv_adj < "+str(adj_pv_max)
comp_filter = "comparison == 'below'"
context_filter = "group.isin(@snvcon_constrained)"
filter_string = " & ".join([spec_filter, pv_filter,
                            comp_filter, context_filter])
threshold_ps_nz_df.query("threshold_index%5 == 0 & "+filter_string)

,group,threshold_index,threshold_quantile,threshold_value,comparison,selected_total,selected_0,selected_1,overall_0,overall_1,frac0_on_selected,frac0_on_unselected,frac0_overall,sensitivity,specificity,test_stat,test_pv,test_pv_adj
9,A>C,5,0.05,-2.294995,below,42803,40641,2162,809111,46938,0.949490,0.944942,0.945169,0.050229,0.953939,16.228655,5.613853e-05,7.583626e-05
409,A>T,5,0.05,-0.440518,below,42826,41574,1252,820228,36287,0.970765,0.956943,0.957634,0.050686,0.965497,191.594512,1.426563e-43,2.601601e-43
609,C>A,5,0.05,-1.727480,below,64041,60266,3775,1190215,90593,0.941053,0.928649,0.929269,0.050635,0.958330,142.431649,7.825277e-33,1.329146e-32
809,C>G,5,0.05,-0.699761,below,55806,51447,4359,1008542,107562,0.921890,0.902666,0.903627,0.051011,0.959475,224.983988,7.401208e-51,1.452985e-50
1009,C>T,5,0.05,-1.572423,below,99720,78254,21466,1525151,469246,0.784737,0.763664,0.764718,0.051309,0.954254,233.814225,8.781177e-53,1.748666e-52
1209,CpG>CpA,5,0.05,-1.531140,below,7734,2187,5547,20576,134095,0.282777,0.125149,0.133031,0.106289,0.958634,1582.850994,0.000000e+00,0.000000e+00
1409,CpG>TpG,5,0.05,-0.627746,below,13148,2394,10754,30987,231954,0.182081,0.114467,0.117848,0.077258,0.953637,549.277754,1.807625e-121,4.656817e-121
1609,G>A,5,0.05,-1.709938,below,86902,68606,18296,1365807,372042,0.789464,0.785731,0.785918,0.050231,0.950823,6.835979,8.933973e-03,1.151859e-02
1809,G>C,5,0.05,-1.013040,below,43543,41492,2051,799893,70959,0.952897,0.916708,0.918518,0.051872,0.971096,723.837506,1.959767e-159,6.131459e-159
